<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install lxml
!pip install parse

In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 25 #limit to check whether they are relative or not

In [58]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 나쁜형사
2 -> 순수의시대
3 -> 신의한수
4 -> 땐뽀걸즈
5 -> 스타쉽 트루퍼스
6 -> 미스터피자
7 -> mp그룹
8 -> 허민
9 -> 정인욱
10 -> 하이라이즈
11 -> 이영자
12 -> 산이
13 -> 이설
14 -> 상장폐지
15 -> 2018 발롱도르
16 -> 키디비
17 -> 손태영
18 -> 노블레스
19 -> 모링가
20 -> 사의찬미 뜻


In [0]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target))
      
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        
        #the representive number should be minimized
        if cnt2 > cnt1:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

In [60]:
print('\n',label,'\n')

score =[0]
score = score*20;

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 4, 5, 6, 6, 8, 8, 10, 11, 12, 1, 6, 15, 16, 17, 18, 19, 20] 



In [61]:
#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  나쁜형사 | 이설
2 .  순수의시대
3 .  미스터피자 | mp그룹 | 상장폐지
4 .  신의한수
5 .  땐뽀걸즈
6 .  허민 | 정인욱
7 .  스타쉽 트루퍼스
8 .  하이라이즈
9 .  이영자
10 .  산이
11 .  2018 발롱도르
12 .  키디비
13 .  손태영
14 .  노블레스
15 .  모링가
16 .  사의찬미 뜻
